# ライブラリ

In [36]:
import pandas
import re
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import numpy as np
import emoji
import statistics

# 分析用DF作成

In [37]:
def make_dataframe(csv):
    """
    データフレーム作るモジュール
    """
    # まずテキストファイルを読み込んでカラム名を変更する
    line_df = pandas.read_csv(csv, sep='\n') 
    line_df = line_df.rename(columns={line_df.columns[0]:'talks'})
    line_df = line_df[1:]

    # 正規表現でトークから日付を取得し新しいカラムとして結合する
    date_list = []
    date_pattern = '(\d+)/(\d+)/\d+\(.?\)'
    for talk in line_df['talks']:
        result = re.match(date_pattern, talk)
        if result:
            date_t = result.group()
            date_list.append(date_t)
        else:
            date_list.append(date_t)

    line_df['date'] = date_list

    # talksにおいて日付のカラムはもう必要ないので、そこがTrueのものをisin関数で消す
    flag = line_df['talks'].isin(line_df['date'])
    line_df = line_df[~flag]
    line_df.dropna(inplace=True)    

    # # talks内を、時間、話者、トーク内容で分ける(ここ結構ガバガバ)
    # # str.splitを使うと、データフレームの列を区切り文字で区切って新しいデータフレームを返す
    time_l = []
    user_l = []
    talk_l = []
    date_l = []
    count = 0
    for date, talk in zip(line_df['date'], line_df['talks']):
        # もし正規表現で時間が取れたらスプリットして3つに分ける
        if(re.match('(\d+):(\d+)', talk)):
            try:
                if(len(talk.split('\t')[0]) == 5):
                    date_l.append(date)
                    time_l.append(talk.split('\t')[0])
                    user_l.append(talk.split('\t')[1])
                    talk_l.append(talk.split('\t')[2])
                    count = count + 1
                else:
                    continue
            except:
                talk_l.append("メッセージの送信取り消し")
                count = count + 1
        else:
            talk_l[count-1] = talk_l[count-1] + talk

    # if(user_l != date_l):
    #     user_l.append(user1)

    line_df = pandas.DataFrame({"date" : date_l,
                            "time" : time_l,
                            "user" : user_l,
                            "talk" : talk_l})

    date_l = []
    for date in line_df['date']:
        date_l.append(date[:-3])
    line_df['date'] = date_l
    
    # datetime型で時間の経過を計算するために、日付と時間をくっつけとく
    line_df['time'] = line_df['date'].str.cat(line_df['time'], sep=' ')
    line_df.drop('date', axis=1, inplace=True)

    # datetime型に変換するところ。indexをリセットしておく。
    line_df['time'] = pandas.to_datetime(line_df['time'], format='%Y/%m/%d %H:%M')
    line_df.reset_index(drop=True, inplace=True)
    
    return line_df

In [38]:
line_df = make_dataframe("[LINE] ryuとのトーク.txt")
line_df

,time,user,talk
0,2021-02-22 11:19:00,Go🕸,こんにちは！
1,2021-02-23 15:00:00,ryu,"""ごめんなさい！全然気づきませんでした！笑"""
2,2021-02-23 15:00:00,ryu,[スタンプ]
3,2021-03-11 17:19:00,Go🕸,無事死にました笑
4,2021-03-11 17:50:00,ryu,順位の話きけました？笑
...,...,...,...
1495,2021-08-04 19:34:00,ryu,いやーー楽しかったｗ
1496,2021-08-04 19:34:00,Go🕸,笑笑
1497,2021-08-04 19:34:00,Go🕸,お疲れ様です笑
1498,2021-08-04 19:34:00,ryu,今日もかてきょですか？


# mecab入れたネガポジチェック

In [39]:
! pwd

/Users/exidea232/Desktop/個人分析/do_you_really_love_me


In [40]:
# 各種ライブラリをインポート
import csv
import time
import matplotlib.pyplot as plt
import MeCab

# MeCabインスタンス作成
tagger = MeCab.Tagger("Desktop/個人分析/do_you_really_love_me/mecab-ipadic-neologd")


# 辞書の読み込み
pn_df = pd.read_csv('/Users/exidea232/Desktop/個人分析/do_you_really_love_me/pn_ja.dic',\
                   sep=':',
                   encoding='shift-jis',
                   names=('Word','Reading','POS', 'PN')
                  )

In [41]:
line_df['time'] = line_df['time'].dt.strftime('%Y/%m/%d')
temp_df = line_df.copy()

# Qiita用に話者隠す

In [42]:
# u1 = temp_df['user'].unique()[0]
# u2 = temp_df['user'].unique()[1]
# for i, name in enumerate(temp_df['user']):
#     if name == u1:
#         temp_df['user'][i] = "me"
#     elif name == u2:
#         temp_df['user'][i] = 'you'

In [43]:
temp_df

,time,user,talk
0,2021/02/22,Go🕸,こんにちは！
1,2021/02/23,ryu,"""ごめんなさい！全然気づきませんでした！笑"""
2,2021/02/23,ryu,[スタンプ]
3,2021/03/11,Go🕸,無事死にました笑
4,2021/03/11,ryu,順位の話きけました？笑
...,...,...,...
1495,2021/08/04,ryu,いやーー楽しかったｗ
1496,2021/08/04,Go🕸,笑笑
1497,2021/08/04,Go🕸,お疲れ様です笑
1498,2021/08/04,ryu,今日もかてきょですか？


In [44]:
pn_df

,Word,Reading,POS,PN
0,優れる,すぐれる,動詞,1.000000
1,良い,よい,形容詞,0.999995
2,喜ぶ,よろこぶ,動詞,0.999979
3,褒める,ほめる,動詞,0.999979
4,めでたい,めでたい,形容詞,0.999645
...,...,...,...,...
55120,ない,ない,助動詞,-0.999997
55121,酷い,ひどい,形容詞,-0.999997
55122,病気,びょうき,名詞,-0.999998
55123,死ぬ,しぬ,動詞,-0.999999


## 1.行ごとに形態素解析

In [46]:
# 並行分散処理かけてあげないと終わらないわ。

In [47]:
ave_polarity = []
tagger = MeCab.Tagger("Desktop/個人分析/do_you_really_love_me/mecab-ipadic-neologd")
for text in temp_df['talk']:
    pn_value = []
    node = tagger.parseToNode(text)
    while node:
        word_value = pn_df[pn_df['Word']==node.surface]['PN'].values
        if len(word_value)>0:
            pn_value.append(word_value[0])
        else:
            pn_value.append(0)
        node = node.next
    ave_polarity.append(sum(pn_value) / len(pn_value))

In [48]:
temp_df['ave_polarity_value'] = ave_polarity

In [49]:
temp_df

,time,user,talk,ave_polarity_value
0,2021/02/22,Go🕸,こんにちは！,0.000000
1,2021/02/23,ryu,"""ごめんなさい！全然気づきませんでした！笑""",0.021693
2,2021/02/23,ryu,[スタンプ],-0.061584
3,2021/03/11,Go🕸,無事死にました笑,0.220980
4,2021/03/11,ryu,順位の話きけました？笑,-0.017787
...,...,...,...,...
1495,2021/08/04,ryu,いやーー楽しかったｗ,0.000000
1496,2021/08/04,Go🕸,笑笑,0.481765
1497,2021/08/04,Go🕸,お疲れ様です笑,0.192706
1498,2021/08/04,ryu,今日もかてきょですか？,0.033063


In [50]:
pn_value = []
text = "並行分散処理かけてあげないと終わらないわ。"
node = tagger.parseToNode(text)
i = 0
while node:
    word_value = pn_df[pn_df['Word']==node.surface]['PN'].values
    pn_value.append(word_value[0]) if len(word_value)>0 else pn_value.append(0)
    print(node.surface, pn_value[i])
    node = node.next
    i = i + 1

 0
並行 0
分散 -0.47483400000000003
処理 -0.698543
かけ 0
て 0
あげ 0
ない -0.9998819999999999
と -0.21579299999999998
終わら 0
ない -0.9998819999999999
わ 0
。 0
 0


In [51]:
pn_df[pn_df['Word']=="分散"]

,Word,Reading,POS,PN
37990,分散,ぶんさん,名詞,-0.474834


In [52]:
temp_df[temp_df['ave_polarity_value']>0]

,time,user,talk,ave_polarity_value
1,2021/02/23,ryu,"""ごめんなさい！全然気づきませんでした！笑""",0.021693
3,2021/03/11,Go🕸,無事死にました笑,0.220980
5,2021/03/11,ryu,"""なんか、今チャット開いたら暴走してますね笑なんか、あったんですか？笑""",0.065798
10,2021/03/11,ryu,笑,0.321177
36,2021/03/11,ryu,それでかじのさんが荒れてるわけですね笑,0.028118
...,...,...,...,...
1486,2021/08/04,ryu,4ぱちだったらなぁ笑,0.192706
1494,2021/08/04,Go🕸,ほうほうほうほう笑笑,0.240882
1496,2021/08/04,Go🕸,笑笑,0.481765
1497,2021/08/04,Go🕸,お疲れ様です笑,0.192706


In [53]:
print("極性がポジティブで登録されている単語数 : ", len(pn_df[pn_df['PN']>0]))
print("極性がネガティブで登録されている単語数 : ", len(pn_df[pn_df['PN']<0]))

極性がポジティブで登録されている単語数 :  5122
極性がネガティブで登録されている単語数 :  49983


In [54]:
len(pn_df[pn_df['PN']<0])

49983

In [55]:
temp_df.groupby("user").sum()

,ave_polarity_value
user,
Go🕸,-2.055841
ryu,-24.438600
ryuがメッセージの送信を取り消しました,-0.124005
